In [1]:
import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import doctable as dt

In [2]:
schema = (
    ('id','integer',dict(primary_key=True, autoincrement=True)),
    ('name','string', dict(nullable=False)),
    ('age','integer'),
    ('is_old', 'boolean'),
)
def new_db():
    db = dt.DocTable2(schema, tabname='mydocuments', verbose=True)
    N = 10
    for i in range(N):
        age = random.random() # number in [0,1]
        is_old = age > 0.5
        db.insert({'name':'user_'+str(i), 'age':age, 'is_old':is_old}, verbose=False)
    return db

db = new_db()
print(db)

DocTable2 Query: SELECT count() AS count_1 
FROM mydocuments
 LIMIT :param_1
<DocTable2::mydocuments ct: 10>


In [3]:
db.select_df(limit=3)

DocTable2 Query: SELECT mydocuments.id, mydocuments.name, mydocuments.age, mydocuments.is_old 
FROM mydocuments
 LIMIT :param_1


,id,name,age,is_old
0,1,user_0,0.083900,False
1,2,user_1,0.494784,False
2,3,user_2,0.847461,True


## Single Update
Update multiple (or single) rows with same values.

In [4]:
db = new_db()
db.select_df(where=db['is_old']==True, limit=3, verbose=False)

,id,name,age,is_old
0,1,user_0,0.968410,True
1,6,user_5,0.972617,True
2,7,user_6,0.578333,True


In [5]:
db = new_db()
db.update({'age':1},where=db['is_old']==True)
db.update({'age':0},where=db['is_old']==False)
db.select_df(limit=3, verbose=False)

DocTable2 Query: UPDATE mydocuments SET age=:age WHERE mydocuments.is_old = true
DocTable2 Query: UPDATE mydocuments SET age=:age WHERE mydocuments.is_old = false


,id,name,age,is_old
0,1,user_0,0,False
1,2,user_1,0,False
2,3,user_2,0,False


## Apply as Map Function
This feature allows you to update columns based on the values of old columns.

In [6]:
db = new_db()
values = {db['name']:db['name']+'th', db['age']:db['age']+1, db['is_old']:True}
db.update(values)
db.select_df(limit=3, verbose=False)

DocTable2 Query: UPDATE mydocuments SET name=(mydocuments.name || :name_1), age=(mydocuments.age + :age_1), is_old=:is_old


,id,name,age,is_old
0,1,user_0th,1.881009,True
1,2,user_1th,1.565073,True
2,3,user_2th,1.713250,True


## Apply as Set of Ordered Map Functions
This is useful for when the updating of one column might change the value of another, depending on the order in which it was applied.

In [7]:
db = new_db()
values = [(db['name'],db['age']-1), (db['age'],db['age']+1),]
db.update(values)
db.select_df(limit=3, verbose=False)

DocTable2 Query: UPDATE mydocuments SET name=(mydocuments.age - :age_1), age=(mydocuments.age + :age_2)


,id,name,age,is_old
0,1,-0.338800414247636,1.661200,True
1,2,-0.137948518648513,1.862051,True
2,3,-0.515398471205426,1.484602,False


## Update Using SQL WHERE String

In [8]:
db = new_db()
db.update({'age':1.00},whrstr='is_old==true')
db.select_df(limit=5, verbose=False)

DocTable2 Query: UPDATE mydocuments SET age=:age WHERE is_old==true


,id,name,age,is_old
0,1,user_0,0.466169,False
1,2,user_1,1.000000,True
2,3,user_2,1.000000,True
3,4,user_3,0.444071,False
4,5,user_4,0.174235,False
